# Merging iTunes Playlists using Pandas

*This notebook originally appeared as a [post](http://lj020326.github.io/blog/2017/12/18/simulating-chutes-and-ladders/) on the blog [Python Automation Journal](http://lj020326.github.io).*

<!-- PELICAN_BEGIN_SUMMARY -->

I recently wanted to see how many songs were common between 2 of my iTunes playlists.

Here I'll take a look at some of the basic questions you can answer with this data.
Later I hope to find the time to dig deeper and ask some more interesting and creative questions – stay tuned!

<!-- PELICAN_END_SUMMARY -->

For those who aren't familiar, this post is composed in the form of a [Jupyter Notebook](https://jupyter.org/), which is an open document format that combines text, code, data, and graphics and is viewable through the web browser – if you have not used it before I encourage you to try it out!

You can download the notebook containing this post [here](http://lj020326.github.io/downloads/notebooks/MergeItunesPlaylists.ipynb), open it with Jupyter, and start asking your own questions of the data.

## Setup notebook output options

Set notebook output formatting options:

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
InteractiveShell.log_level = 'INFO'

# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))


## Import standard libraries

Some standard Python package imports:

In [4]:
%matplotlib inline
import pandas as pd
import numpy as np
import logging
import sys

from collections import OrderedDict

log = logging.getLogger()
log.handlers = []
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
log.setLevel(logging.INFO)
# log.setLevel(logging.DEBUG)
log.addHandler(ch)

pd.set_option('display.width',1000)
# pd.set_option('max_colwidth',200)
pd.reset_option('max_colwidth')



## Import Itunes Playlist Data

We'll start by exporting the itunes playlist data (for howto export itunes playlists [itunes](https://support.apple.com/kb/PH19491?locale=en_US)).
I chose the text export option rather than xml.

After exporting, I opened and saved the file in excel.
I now have 2 playlist excel files as seen in the example below.

Note that I could directly import the text formatted file.
I use excel here to simply demonstrate that excel based imports will also work.

We now load the 2 playlists into list_of_playlists.


In [5]:

all_files = ['data/Top80_sPop&Rock.xlsx','data/TopAlt&Electronic.xlsx']
list_of_playlists= []
for file_ in all_files:
    xls_file = pd.ExcelFile(file_)
    df = xls_file.parse(0)  ## load first sheet - there should only be one
#     df = pd.read_csv(file_,index_col=None, header=0)
    log.info("## %s: df.head()=" % file_)
    df.head()
    log.info("## %s: df.shape=%s" % (file_, str(df.shape)))
    list_of_playlists.append(df)


2018-04-24 14:06:03,348 - INFO - ## data/Top80_sPop&Rock.xlsx: df.head()=


,Name,Artist,Composer,Album,Grouping,Work,Movement Number,Movement Count,Movement Name,Genre,...,Volume Adjustment,Kind,Equalizer,Comments,Plays,Last Played,Skips,Last Skipped,My Rating,Location
0,Just The Way You Are,Billy Joel,Billy Joel,Greatest Hits Vol.I,NaN,NaN,NaN,NaN,NaN,"Pop, 80s",...,NaN,MPEG audio file,NaN,NaN,154.0,2018-04-07 09:49:00,2.0,2017-12-27 10:16:00,100,C:\Users\Lee\itunes\Music\Billy Joel\Greatest ...
1,She's Got A Way,Billy Joel,Billy Joel,Greatest Hits Vol.II,NaN,NaN,NaN,NaN,NaN,"Pop, 80s",...,NaN,MPEG audio file,NaN,NaN,144.0,2018-03-25 14:02:00,3.0,2018-01-20 08:18:00,100,C:\Users\Lee\itunes\Music\Billy Joel\Greatest ...
2,Thunderstruck,Brian Johnson,NaN,The Very Best of AC/DC,NaN,NaN,NaN,NaN,NaN,"Hard Rock, 80s",...,NaN,MPEG audio file,NaN,NaN,96.0,2018-02-07 11:01:00,7.0,2018-01-08 18:35:00,100,C:\Users\Lee\itunes\Music\Compilations\The Ver...
3,Don't Stop Believin',Journey,Jonathan Cain/Neal Schon/Steve Perry,Greatest Hits (Limited Gold Edition),NaN,NaN,NaN,NaN,NaN,"Pop, 80s",...,NaN,MPEG audio file,NaN,Encoded from Original CD album. Lame v3.98.2 ...,86.0,2018-02-09 07:07:00,6.0,2018-01-06 12:45:00,100,C:\Users\Lee\itunes\Music\Journey\Greatest Hit...
4,Fade To Grey,Visage,NaN,"New Wave Hits Of The '80s, Vol. 3",NaN,NaN,NaN,NaN,NaN,"Rock, 80s, Alternative",...,NaN,MPEG audio file,NaN,NaN,62.0,2018-04-09 19:23:00,NaN,NaT,100,C:\Users\Lee\itunes\Music\Visage\New Wave Hits...


2018-04-24 14:06:03,391 - INFO - ## data/Top80_sPop&Rock.xlsx: df.shape=(678, 31)
2018-04-24 14:06:03,461 - INFO - ## data/TopAlt&Electronic.xlsx: df.head()=


,Name,Artist,Composer,Album,Grouping,Work,Movement Number,Movement Count,Movement Name,Genre,...,Volume Adjustment,Kind,Equalizer,Comments,Plays,Last Played,Skips,Last Skipped,My Rating,Location
0,The Walker,Fitz & The Tantrums,NaN,More Than Just a Dream (Deluxe Version),NaN,NaN,NaN,NaN,NaN,Alternative,...,NaN,MPEG audio file,NaN,NaN,176.0,2017-12-05 10:46:00,10.0,2018-01-08 19:45:00,100,C:\Users\Lee\itunes\Music\Fitz & The Tantrums\...
1,Jack The Ripper,Morrissey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Alternative, Punk",...,NaN,MPEG audio file,NaN,NaN,436.0,2018-03-12 19:59:00,13.0,2017-09-28 18:59:00,100,C:\Users\Lee\itunes\Music\Morrissey\Unknown Al...
2,Heathens,Twenty One Pilots,NaN,Suicide Squad: The Album,NaN,NaN,NaN,NaN,NaN,"Indie Rock, Alternative",...,NaN,MPEG audio file,NaN,NaN,176.0,2017-11-21 14:29:00,NaN,NaT,100,C:\Users\Lee\itunes\Music\Twenty One Pilots\Su...
3,Fade To Grey,Visage,NaN,"New Wave Hits Of The '80s, Vol. 3",NaN,NaN,NaN,NaN,NaN,"Rock, 80s, Alternative",...,NaN,MPEG audio file,NaN,NaN,62.0,2018-04-09 19:23:00,NaN,NaT,100,C:\Users\Lee\itunes\Music\Visage\New Wave Hits...
4,Take A Chance On Me,ABBA,NaN,The Essential Collection (CD 1/2),NaN,NaN,NaN,NaN,NaN,Disco/Dance/Electronica/Pop,...,NaN,MPEG audio file,NaN,NaN,96.0,2017-11-21 14:37:00,23.0,2017-10-30 18:33:00,80,C:\Users\Lee\itunes\Music\ABBA\The Essential C...


2018-04-24 14:06:03,503 - INFO - ## data/TopAlt&Electronic.xlsx: df.shape=(186, 31)


## Vertically Stack the Playlist Data

For our first example - I will load the playlists in list_of_playlists into dataframes and stack vertically.

When we concatenate DataFrames, we need to specify the axis. axis=0 tells pandas to stack the second DataFrame under the first one. It will automatically detect whether the column names are the same and will stack accordingly. To stack the data vertically, we need to make sure we have the same columns and associated column format in both datasets. 


In [6]:

# Stack the DataFrames on top of each other
vertical_stack = pd.concat(list_of_playlists, axis=0)

log.info("## vertical_stack.head()=")
vertical_stack.head()
log.info("vertical_stack.shape=%s" % str(vertical_stack.shape))


2018-04-24 14:06:09,217 - INFO - ## vertical_stack.head()=


,Name,Artist,Composer,Album,Grouping,Work,Movement Number,Movement Count,Movement Name,Genre,...,Volume Adjustment,Kind,Equalizer,Comments,Plays,Last Played,Skips,Last Skipped,My Rating,Location
0,Just The Way You Are,Billy Joel,Billy Joel,Greatest Hits Vol.I,NaN,NaN,NaN,NaN,NaN,"Pop, 80s",...,NaN,MPEG audio file,NaN,NaN,154.0,2018-04-07 09:49:00,2.0,2017-12-27 10:16:00,100,C:\Users\Lee\itunes\Music\Billy Joel\Greatest ...
1,She's Got A Way,Billy Joel,Billy Joel,Greatest Hits Vol.II,NaN,NaN,NaN,NaN,NaN,"Pop, 80s",...,NaN,MPEG audio file,NaN,NaN,144.0,2018-03-25 14:02:00,3.0,2018-01-20 08:18:00,100,C:\Users\Lee\itunes\Music\Billy Joel\Greatest ...
2,Thunderstruck,Brian Johnson,NaN,The Very Best of AC/DC,NaN,NaN,NaN,NaN,NaN,"Hard Rock, 80s",...,NaN,MPEG audio file,NaN,NaN,96.0,2018-02-07 11:01:00,7.0,2018-01-08 18:35:00,100,C:\Users\Lee\itunes\Music\Compilations\The Ver...
3,Don't Stop Believin',Journey,Jonathan Cain/Neal Schon/Steve Perry,Greatest Hits (Limited Gold Edition),NaN,NaN,NaN,NaN,NaN,"Pop, 80s",...,NaN,MPEG audio file,NaN,Encoded from Original CD album. Lame v3.98.2 ...,86.0,2018-02-09 07:07:00,6.0,2018-01-06 12:45:00,100,C:\Users\Lee\itunes\Music\Journey\Greatest Hit...
4,Fade To Grey,Visage,NaN,"New Wave Hits Of The '80s, Vol. 3",NaN,NaN,NaN,NaN,NaN,"Rock, 80s, Alternative",...,NaN,MPEG audio file,NaN,NaN,62.0,2018-04-09 19:23:00,NaN,NaT,100,C:\Users\Lee\itunes\Music\Visage\New Wave Hits...


2018-04-24 14:06:09,256 - INFO - vertical_stack.shape=(864, 31)


## Merge into one dataframe based on composite keys.

We can also merge the list dataframe into one dataframe based on a common unique key - or multiple keys.  
Note that I refer to the term "composite key" when there multiple keys used to identify a record.


In [7]:
df_playlist1=list_of_playlists[0]
df_playlist2=list_of_playlists[1]

merged_df = pd.merge(df_playlist1, df_playlist2, on=['Artist','Name'])

log.info("## merged_df.head()=")
merged_df.head()
log.info("merged_df.shape=%s" % str(merged_df.shape))


2018-04-24 14:06:12,375 - INFO - ## merged_df.head()=


,Name,Artist,Composer_x,Album_x,Grouping_x,Work_x,Movement Number_x,Movement Count_x,Movement Name_x,Genre_x,...,Volume Adjustment_y,Kind_y,Equalizer_y,Comments_y,Plays_y,Last Played_y,Skips_y,Last Skipped_y,My Rating_y,Location_y
0,Fade To Grey,Visage,NaN,"New Wave Hits Of The '80s, Vol. 3",NaN,NaN,NaN,NaN,NaN,"Rock, 80s, Alternative",...,NaN,MPEG audio file,NaN,NaN,62.0,2018-04-09 19:23:00,NaN,NaT,100,C:\Users\Lee\itunes\Music\Visage\New Wave Hits...
1,I Could Be Happy,Altered Images,NaN,"New Wave Hits Of The '80s, Vol. 04",NaN,NaN,NaN,NaN,NaN,"Pop, New Wave, 80s, Alternative",...,NaN,MPEG audio file,NaN,NaN,66.0,2018-04-11 17:44:00,1.0,2018-01-06 20:21:00,80,C:\Users\Lee\itunes\Music\Altered Images\New W...
2,Dance This Mess Around,The B-52's,Cindy Wilson/Fred Schneider/Kate Pierson/Keith...,B-52's,NaN,NaN,NaN,NaN,NaN,"Alternative, Punk, 80s",...,NaN,MPEG audio file,NaN,Black1000,151.0,2018-04-10 15:09:00,6.0,2018-03-07 11:58:00,80,C:\Users\Lee\itunes\Music\The B-52's\B-52's\03...
3,Fight For Your Right,Beastie Boys,NaN,The Best 80s Songs,NaN,.,NaN,NaN,NaN,"80s, Pop, Alternative",...,NaN,MPEG audio file,NaN,NaN,53.0,2018-04-09 20:18:00,1.0,2018-01-02 01:37:00,80,C:\Users\Lee\itunes\Music\Compilations\The Bes...
4,Just What I Needed,The Cars,Ric Ocasek,Complete Greatest Hits,NaN,NaN,NaN,NaN,NaN,"Pop, Electronic, 80s",...,NaN,MPEG audio file,NaN,NaN,112.0,2018-01-08 12:44:00,4.0,2018-01-06 22:59:00,80,C:\Users\Lee\itunes\Music\The Cars\Complete Gr...


2018-04-24 14:06:12,414 - INFO - merged_df.shape=(72, 60)


By default, the Pandas merge operation acts with an “inner” merge. An inner merge, (or inner join) keeps only the common values in both the left and right dataframes for the result.

Note that the # of columns minus the specified common keys has doubled.

The merged dataframe preserves the original data columns from each of the datasets.

The merged results has an appended suffix for each set of columns, "_x" and "_y" in the prior example. 


## Inner, Left, and right merge types

There are three different types of merges available in Pandas. These merge types are common across most database and data-orientated languages (SQL, R, SAS) and are typically referred to as “joins”. If you don’t know them, learn them now.

1. Inner Merge / Inner join – The default Pandas behaviour, only keep rows where the merge “on” value exists in both the left and right dataframes.
2. Left Merge / Left outer join – (aka left merge or left join) Keep every row in the left dataframe. Where there are missing values of the “on” variable in the right dataframe, add empty / NaN values in the result.
3. Right Merge / Right outer join – (aka right merge or right join) Keep every row in the right dataframe. Where there are missing values of the “on” variable in the left column, add empty / NaN values in the result.
4. Outer Merge / Full outer join – A full outer join returns all the rows from the left dataframe, all the rows from the right dataframe, and matches up rows where possible, with NaNs elsewhere.

The merge type to use is specified using the “how” parameter in the merge command, taking values “left”, “right”, “inner” (default), or “outer”.

Venn diagrams are commonly used to exemplify the different merge and join types. See this example from [Stack overflow](http://stackoverflow.com/questions/38549/what-is-the-difference-between-inner-join-and-outer-join):

<img src="/images/join-types-merge-names.jpg" alt='[img: Venn Diagram for merge types]'>


## Reduce into one dataframe based on the composite key

We can also use reduce to merge the list dataframe into one.

This gets the same results as the merge in the last example.

In [8]:

import functools
merged_df = functools.reduce(lambda left,right: pd.merge(left,right,on=['Artist','Name']), list_of_playlists)

log.info("## merged_df.head()=")
merged_df.head()
log.info("merged_df.shape=%s" % str(merged_df.shape))


2018-04-24 14:06:17,095 - INFO - ## merged_df.head()=


,Name,Artist,Composer_x,Album_x,Grouping_x,Work_x,Movement Number_x,Movement Count_x,Movement Name_x,Genre_x,...,Volume Adjustment_y,Kind_y,Equalizer_y,Comments_y,Plays_y,Last Played_y,Skips_y,Last Skipped_y,My Rating_y,Location_y
0,Fade To Grey,Visage,NaN,"New Wave Hits Of The '80s, Vol. 3",NaN,NaN,NaN,NaN,NaN,"Rock, 80s, Alternative",...,NaN,MPEG audio file,NaN,NaN,62.0,2018-04-09 19:23:00,NaN,NaT,100,C:\Users\Lee\itunes\Music\Visage\New Wave Hits...
1,I Could Be Happy,Altered Images,NaN,"New Wave Hits Of The '80s, Vol. 04",NaN,NaN,NaN,NaN,NaN,"Pop, New Wave, 80s, Alternative",...,NaN,MPEG audio file,NaN,NaN,66.0,2018-04-11 17:44:00,1.0,2018-01-06 20:21:00,80,C:\Users\Lee\itunes\Music\Altered Images\New W...
2,Dance This Mess Around,The B-52's,Cindy Wilson/Fred Schneider/Kate Pierson/Keith...,B-52's,NaN,NaN,NaN,NaN,NaN,"Alternative, Punk, 80s",...,NaN,MPEG audio file,NaN,Black1000,151.0,2018-04-10 15:09:00,6.0,2018-03-07 11:58:00,80,C:\Users\Lee\itunes\Music\The B-52's\B-52's\03...
3,Fight For Your Right,Beastie Boys,NaN,The Best 80s Songs,NaN,.,NaN,NaN,NaN,"80s, Pop, Alternative",...,NaN,MPEG audio file,NaN,NaN,53.0,2018-04-09 20:18:00,1.0,2018-01-02 01:37:00,80,C:\Users\Lee\itunes\Music\Compilations\The Bes...
4,Just What I Needed,The Cars,Ric Ocasek,Complete Greatest Hits,NaN,NaN,NaN,NaN,NaN,"Pop, Electronic, 80s",...,NaN,MPEG audio file,NaN,NaN,112.0,2018-01-08 12:44:00,4.0,2018-01-06 22:59:00,80,C:\Users\Lee\itunes\Music\The Cars\Complete Gr...


2018-04-24 14:06:17,132 - INFO - merged_df.shape=(72, 60)


## Example of outer merge / full outer join

Finally, we will perform an outer merge using Pandas, also referred to as a “full outer join” or just “outer join”. An outer join can be seen as a combination of left and right joins, or the opposite of an inner join. In outer joins, every row from the left and right dataframes is retained in the result, with NaNs where there are no matched join variables.

As such, we would expect the results to have the same number of rows as there are distinct values of “use_id” between user_device and user_usage, i.e. every join value from the left dataframe will be in the result along with every value from the right dataframe, and they’ll be linked where possible.


In [9]:
df_playlist1=list_of_playlists[0]
df_playlist2=list_of_playlists[1]

merged_df = pd.merge(df_playlist1, 
                     df_playlist2, 
                     on=['Artist','Name'],
                     how='outer')

log.info("## merged_df.head()=")
merged_df.head()
log.info("merged_df.shape=%s" % str(merged_df.shape))


2018-04-24 14:06:20,719 - INFO - ## merged_df.head()=


,Name,Artist,Composer_x,Album_x,Grouping_x,Work_x,Movement Number_x,Movement Count_x,Movement Name_x,Genre_x,...,Volume Adjustment_y,Kind_y,Equalizer_y,Comments_y,Plays_y,Last Played_y,Skips_y,Last Skipped_y,My Rating_y,Location_y
0,Just The Way You Are,Billy Joel,Billy Joel,Greatest Hits Vol.I,NaN,NaN,NaN,NaN,NaN,"Pop, 80s",...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN
1,She's Got A Way,Billy Joel,Billy Joel,Greatest Hits Vol.II,NaN,NaN,NaN,NaN,NaN,"Pop, 80s",...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN
2,Thunderstruck,Brian Johnson,NaN,The Very Best of AC/DC,NaN,NaN,NaN,NaN,NaN,"Hard Rock, 80s",...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN
3,Don't Stop Believin',Journey,Jonathan Cain/Neal Schon/Steve Perry,Greatest Hits (Limited Gold Edition),NaN,NaN,NaN,NaN,NaN,"Pop, 80s",...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN
4,Fade To Grey,Visage,NaN,"New Wave Hits Of The '80s, Vol. 3",NaN,NaN,NaN,NaN,NaN,"Rock, 80s, Alternative",...,NaN,MPEG audio file,NaN,NaN,62.0,2018-04-09 19:23:00,NaN,NaT,100.0,C:\Users\Lee\itunes\Music\Visage\New Wave Hits...


2018-04-24 14:06:20,756 - INFO - merged_df.shape=(797, 60)


We noted in our prior example that the vertically stacked dataframe had a rowcount of 864 and we noted that the inner data set found from the initial merge count is 72 rows.

In theory the rowcount of the outer join should be the total count minus the intersection - so 864 - 72 = 792.

However, the count is off by 5. 

Something is off.  

My instincts are that the data sets are not truely unique based on the composite key that we used.  This would lead to mismatched counts between the union count and the merged data set count.

We can perform a simple check to validate if our instincts are correct.
We now find out how many duplicate records exist based on the composite key we used in the first playlist dataset.


In [10]:

df_dupes1 = df_playlist1[df_playlist1.duplicated(['Artist','Name'], keep=False)]
log.info("df_dupes1.shape=%s" % str(df_dupes1.shape))

df_dupes1.sort_values(['Artist','Name']).head(10)


2018-04-24 14:06:23,911 - INFO - df_dupes1.shape=(76, 31)


,Name,Artist,Composer,Album,Grouping,Work,Movement Number,Movement Count,Movement Name,Genre,...,Volume Adjustment,Kind,Equalizer,Comments,Plays,Last Played,Skips,Last Skipped,My Rating,Location
122,Halloween,Dead Kennedys,NaN,Just Can't Get Enough: New Wa,NaN,NaN,NaN,NaN,NaN,"Alternative, 80s",...,NaN,MPEG audio file,NaN,NaN,NaN,NaT,4.0,2018-03-19 08:46:00,60,C:\Users\Lee\itunes\Music\Dead Kennedys\Just C...
310,Halloween,Dead Kennedys,D.H. Peligro/Dead Kennedys/East Bay Ray/Jello ...,Milking The Sacred Cow,NaN,NaN,NaN,NaN,NaN,"Punk Rock, 80s",...,NaN,MPEG audio file,NaN,NaN,NaN,NaT,1.0,2018-01-11 23:59:00,40,C:\Users\Lee\itunes\Music\Dead Kennedys\Milkin...
20,Enjoy The Silence,Depeche Mode,NaN,"Best Of Depeche Mode, Vol. 1",NaN,NaN,NaN,NaN,NaN,"Pop, 80s",...,NaN,MPEG audio file,NaN,Track 4,40.0,2018-04-10 19:19:00,1.0,2018-01-14 10:37:00,80,C:\Users\Lee\itunes\Music\Depeche Mode\Best Of...
127,Enjoy The Silence,Depeche Mode,Martin L. Gore,Violator,NaN,NaN,NaN,NaN,NaN,"Pop, 80s, Alternative",...,NaN,MPEG audio file,NaN,Sire / Reprise 9 26081-2,27.0,2018-04-09 19:49:00,2.0,2018-02-23 13:33:00,60,C:\Users\Lee\itunes\Music\Depeche Mode\Violato...
29,The Final Countdown,Europe,NaN,101 Sporting Anthems,NaN,NaN,NaN,NaN,NaN,"Hard Rock, 80s",...,NaN,MPEG audio file,NaN,NaN,86.0,2016-04-29 15:29:00,5.0,2015-09-28 19:58:00,80,C:\Users\Lee\itunes\Music\Various Artists\101 ...
358,The Final Countdown,Europe,NaN,The Best 80s Songs,NaN,.,NaN,NaN,NaN,"80s, Pop",...,NaN,MPEG audio file,NaN,NaN,3.0,2018-01-02 17:47:00,2.0,2018-01-06 21:49:00,40,C:\Users\Lee\itunes\Music\Compilations\The Bes...
36,Down Under,Men At Work,NaN,The Best 80s Songs,NaN,.,NaN,NaN,NaN,"80s, Pop",...,NaN,MPEG audio file,NaN,NaN,2.0,2018-01-02 19:24:00,1.0,2018-01-02 01:44:00,80,C:\Users\Lee\itunes\Music\Compilations\The Bes...
37,Down Under,Men At Work,"Colin Hay, Roy Strykert","Like, Omigod! The 80s Pop Culture Box (Totally...",NaN,NaN,NaN,NaN,NaN,"Pop, 80s",...,NaN,MPEG audio file,NaN,NaN,49.0,2018-04-11 18:47:00,3.0,2018-01-17 18:53:00,80,"C:\Users\Lee\itunes\Music\Men At Work\Like, Om..."
45,Cum On Feel The Noize,Quiet Riot,NaN,101 Sporting Anthems,NaN,NaN,NaN,NaN,NaN,"Hard Rock, 80s",...,NaN,MPEG audio file,NaN,NaN,79.0,2016-04-29 16:44:00,5.0,2018-01-08 15:01:00,80,C:\Users\Lee\itunes\Music\Various Artists\101 ...
485,Cum On Feel The Noize,Quiet Riot,"Noddy Holder, Jim Lea","Like, Omigod! The 80s Pop Culture Box (Totally...",NaN,NaN,NaN,NaN,NaN,"Pop, 80s",...,NaN,MPEG audio file,NaN,NaN,3.0,2018-03-30 10:27:00,2.0,2018-01-09 18:04:00,40,"C:\Users\Lee\itunes\Music\Compilations\Like, O..."


There 76 duplicate entries in the set based on the composite key we used.

Looking at the data, it seems it contains information for the actual digital files stored on disk.

For our purpose of comparing the logical music tracks between the 2 playlists, we can safely remove that physical file related information to form a new logical representation of the playlist based on a subset of the data.

For now, we will simply strip out the duplicate records from the playlist based on original composite key and keep the first record in each case.



In [11]:
df_playlist1_uniq=df_playlist1.drop_duplicates(subset=['Artist', 'Name'])

log.info("df_playlist1_uniq.shape=%s" % str(df_playlist1_uniq.shape))


2018-04-24 14:06:29,114 - INFO - df_playlist1_uniq.shape=(634, 31)


## Enhance the dataset load to exclude duplicates

We now update the original load logic to exclude duplicates and rerun the merge to see if the set union set count agrees with the merged dataset count.


In [12]:

all_files = ['data/Top80_sPop&Rock.xlsx','data/TopAlt&Electronic.xlsx']
list_of_playlists= []
for file_ in all_files:
    xls_file = pd.ExcelFile(file_)
    df = xls_file.parse(0)  ## load first sheet - there should only be one
    
    df.drop_duplicates(subset=['Artist', 'Name'], inplace=True)
    log.info("## %s: df.shape=%s" % (file_, str(df.shape)))

    list_of_playlists.append(df)

inner_df = reduce(lambda left,right: pd.merge(left,right,on=['Artist','Name'], how='inner'), list_of_playlists)
merged_df = reduce(lambda left,right: pd.merge(left,right,on=['Artist','Name'], how='outer'), list_of_playlists)

log.info("inner_df.shape=%s" % str(inner_df.shape))
log.info("merged_df.shape=%s" % str(merged_df.shape))

df1_cnt=list_of_playlists[0].shape[0]
df2_cnt=list_of_playlists[1].shape[0]
df_inner_cnt=inner_df.shape[0]
df_union_cnt=df1_cnt+df2_cnt-df_inner_cnt
df_merged_cnt=merged_df.shape[0]

log.info("Rows in dataset1=%s" % df1_cnt)
log.info("Rows in dataset2=%s" % df2_cnt)
log.info("Rows in intersection=%s" % df_inner_cnt)
log.info("Rows in union=%s" % df_union_cnt)
log.info("(Rows in union==Rows in merged) = %s" % (df_union_cnt==df_merged_cnt))


2018-04-24 14:06:31,750 - INFO - ## data/Top80_sPop&Rock.xlsx: df.shape=(634, 31)
2018-04-24 14:06:31,811 - INFO - ## data/TopAlt&Electronic.xlsx: df.shape=(185, 31)
2018-04-24 14:06:31,831 - INFO - inner_df.shape=(67, 60)
2018-04-24 14:06:31,834 - INFO - merged_df.shape=(752, 60)
2018-04-24 14:06:31,836 - INFO - Rows in dataset1=634
2018-04-24 14:06:31,841 - INFO - Rows in dataset2=185
2018-04-24 14:06:31,842 - INFO - Rows in intersection=67
2018-04-24 14:06:31,845 - INFO - Rows in union=752
2018-04-24 14:06:31,846 - INFO - (Rows in union==Rows in merged) = True


So the data set union count agrees with the merged dataframe count!
